# YOFO - You Only Fisheye Once

Pipeline for Localize fisheye image using global descriptor

In [1]:
# Check if we have NVIDIA Driver installed !
!nvidia-smi

Wed Mar 13 16:33:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:08:00.0 Off |                  N/A |
| 54%   60C    P2             260W / 420W |   7061MiB / 24576MiB |     95%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Import Stuff

In [2]:
# Imports
# Date Time
from dateutil import tz
from datetime import datetime

# Libraries
import sys
import os
import copy
import json
import numpy as np
import shutil
import PIL

# Scikit-Learn
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets, metrics

# PYTORCH !
import torch
import torchinfo
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# Visualization
%matplotlib widget
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
from torch.utils.tensorboard import SummaryWriter

# Path for Import Internal Modules
sys.path.append("/workspace/fisheye-vpr")


2024-03-13 16:33:04.104867: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 16:33:05.418855: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Fix Seed for Regeneration

In [3]:
## Fix Seed for Regeneration
torch.manual_seed(777)

---

### Import Model

In [4]:
# Main Model
from models.vpr_model import VPRModel

# Feature Extractor
from models.encoders.resnet18 import Resnet18Encoder
from models.encoders.vgg16 import VGG16Encoder

# Clustering
from models.clustering.netvlad import NetVLAD

# Loss
from models.loss_function.HardTripletLoss import HardTripletLoss

# Trainer
from util.train import Trainer

### Dataset Selection

In [5]:
from dataloaders.isl2_3places_fisheye import ISL2_3Places_Fisheye_Dataset
dataset = ISL2_3Places_Fisheye_Dataset()

---

### Model Assembling

In [6]:
netvlad_config = {
    "num_clusters": 20,
    "desc_dim": 512,            # Up to the Feature Extraction Module
    "alpha": 100.0,
    "normalize_input": True,
}
model = VPRModel(
    feature_extractor=Resnet18Encoder(fine_tuning=False),
    clustering=NetVLAD(**netvlad_config)
)
model = model.to("cuda")

[RESNET18] Output Dim Size: 512
NetVLAD Module initialized !
 - self.num_clusters : 20
 - self.dim : 512
 - self.alpha : 100.0
 - self.normalize_input : True


### Criterion (Loss Function)

In [7]:
criterion = HardTripletLoss()

---

### Trainer

In [8]:
# Configuration
config = {
    "run_name": "default",
    "start_time": "",
    "training_epoch": 20
}

In [9]:
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.0001,
    weight_decay=0.001,
    momentum=0.9
)
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=5,
    gamma=0.5
)

In [10]:
trainer = Trainer(
    dataset,
    model,
    criterion,
    optimizer,
    scheduler,
    config,
)

Run Name : resnet18_NetVLAD20_ISL2_LabLiftToiletFisheye_13-Mar-2024_23-33-08


In [11]:
trainer.train()

==> Epoch 0 Training: 100%|██████████| 29/29 [00:06<00:00,  4.30 batch/s, batch_idx=28, epoch_idx=0, loss=0.127]


Validating epoch 0


==> Epoch 1 Training: 100%|██████████| 29/29 [00:05<00:00,  5.09 batch/s, batch_idx=28, epoch_idx=1, loss=0.121]


Validating epoch 1


==> Epoch 2 Training: 100%|██████████| 29/29 [00:05<00:00,  5.07 batch/s, batch_idx=28, epoch_idx=2, loss=0.0404]


...

==> Epoch 18 Training: 100%|██████████| 29/29 [00:05<00:00,  4.98 batch/s, batch_idx=28, epoch_idx=18, loss=0.118]


Validating epoch 18


==> Epoch 19 Training: 100%|██████████| 29/29 [00:05<00:00,  5.12 batch/s, batch_idx=28, epoch_idx=19, loss=0.0628]


Validating epoch 19


Model Training: 100%|██████████| 20/20 [02:09<00:00,  6.50s/epoch]
